In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
intubated_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/intubation_trigger/data/Mastertabelle_filtered.xlsx'
non_intubated_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/intubation_trigger/data/not_intubated/St_GallenBernZürichNicht-intubiert.xlsx'

In [ ]:
intubated_data = pd.read_excel(intubated_data_path)
non_intubated_data = pd.read_excel(non_intubated_path)

In [ ]:
intubated_data = intubated_data[intubated_data.Comment != 'Reanimation']

In [ ]:
intubated_data['Intubated'] = 1
non_intubated_data['Intubated'] = 0

In [ ]:
intubated_data.head()

In [ ]:
intubated_data['Einteilung (Hauptdiagnose)'].value_counts()

In [ ]:
spo2_df = pd.concat([intubated_data[['SPO2 (Erstbefund)', 'Intubated', 'Einteilung (Hauptdiagnose)']], non_intubated_data[['SPO2 (Erstbefund)', 'Intubated', 'Einteilung (Hauptdiagnose)']]]).reset_index()

In [ ]:
# plot distribution of SPO2 intubated vs non-intubated (with two subplots for Trauma and Non-Trauma)
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
sns.histplot(data=spo2_df[spo2_df['Einteilung (Hauptdiagnose)'] == 'Unfall'], x='SPO2 (Erstbefund)', hue='Intubated', kde=True, ax=ax[0], discrete=True)
ax[0].set_title('Trauma')

sns.histplot(data=spo2_df[spo2_df['Einteilung (Hauptdiagnose)'] == 'Krankheit'], x='SPO2 (Erstbefund)', hue='Intubated', kde=True, ax=ax[1], discrete=True)
ax[1].set_title('Non-Trauma')

plt.show()

In [ ]:
# find a cutoff for spo2 to predict intubation in trauma and non trauma patients 
# compute c statistic for every cutoff

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

trauma_roc_auc_scores = []
trauma_fpr = []
trauma_tpr = []
trauma_thresholds = []

non_trauma_roc_auc_scores = []
non_trauma_fpr = []
non_trauma_tpr = []
non_trauma_thresholds = []

for i in range(50, 100):
    spo2_df['Intubated_pred'] = spo2_df['SPO2 (Erstbefund)'] < i
    trauma_roc_auc_scores.append(roc_auc_score(spo2_df[spo2_df['Einteilung (Hauptdiagnose)'] == 'Unfall']['Intubated'], spo2_df[spo2_df['Einteilung (Hauptdiagnose)'] == 'Unfall']['Intubated_pred']))
    fpr, tpr, thresholds = roc_curve(spo2_df[spo2_df['Einteilung (Hauptdiagnose)'] == 'Unfall']['Intubated'], spo2_df[spo2_df['Einteilung (Hauptdiagnose)'] == 'Unfall']['Intubated_pred'])
    trauma_fpr.append(fpr)
    trauma_tpr.append(tpr)
    trauma_thresholds.append(thresholds)

    non_trauma_roc_auc_scores.append(roc_auc_score(spo2_df[spo2_df['Einteilung (Hauptdiagnose)'] == 'Krankheit']['Intubated'], spo2_df[spo2_df['Einteilung (Hauptdiagnose)'] == 'Krankheit']['Intubated_pred']))
    fpr, tpr, thresholds = roc_curve(spo2_df[spo2_df['Einteilung (Hauptdiagnose)'] == 'Krankheit']['Intubated'], spo2_df[spo2_df['Einteilung (Hauptdiagnose)'] == 'Krankheit']['Intubated_pred'])
    non_trauma_fpr.append(fpr)
    non_trauma_tpr.append(tpr)
    non_trauma_thresholds.append(thresholds)
    



In [ ]:
# plot ROC AUC scores
plt.figure(figsize=(10, 5))

ax = sns.lineplot(x=np.arange(50, 100, 1), y=trauma_roc_auc_scores, label='Trauma', color='red')
ax = sns.lineplot(x=np.arange(50, 100, 1), y=non_trauma_roc_auc_scores, label='Non Trauma', color='blue', ax=ax)

plt.xlabel('SpO2 threshold')
plt.ylabel('ROC AUC score')
plt.title('ROC AUC score for different SpO2 thresholds')

plt.show()

In [ ]:
# overall roc auc score for spo2 as a predictor of intubation in trauma and non trauma patients
spo2_df.dropna(inplace=True)
overall_rocauc = roc_auc_score(spo2_df['Intubated'], spo2_df['SPO2 (Erstbefund)'])
trauma_rocauc = roc_auc_score(spo2_df[spo2_df['Einteilung (Hauptdiagnose)'] == 'Unfall']['Intubated'], spo2_df[spo2_df['Einteilung (Hauptdiagnose)'] == 'Unfall']['SPO2 (Erstbefund)'])
non_trauma_rocauc = roc_auc_score(spo2_df[spo2_df['Einteilung (Hauptdiagnose)'] == 'Krankheit']['Intubated'], spo2_df[spo2_df['Einteilung (Hauptdiagnose)'] == 'Krankheit']['SPO2 (Erstbefund)'])

print(f'Overall ROC AUC score: {overall_rocauc}')
print(f'Trauma ROC AUC score: {trauma_rocauc}')
print(f'Non Trauma ROC AUC score: {non_trauma_rocauc}')
